In [1]:
from __future__ import division, print_function

In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline
%load_ext wurlitzer


Populating the interactive namespace from numpy and matplotlib


In [9]:
import time
import numpy as np

from director import viewerclient as vc
from irispy import Polyhedron
import boxatlas as box
from contactstabilization import BoxAtlasContactStabilization

In [10]:
vis = vc.Visualizer()

In [21]:
surfaces = [
    box.Surface(Polyhedron.fromBounds([1.2, 0], [1.2, 2]),
            Polyhedron(np.array([[1, -1], [1, 1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2, 0], [2, 0]), 
            Polyhedron(np.array([[-2, -1], [2, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2, 0], [2, 0]), 
            Polyhedron(np.array([[-2, -1], [2, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-1.2, 0], [-1.2, 2]),
            Polyhedron(np.array([[-1, -1], [-1, 1]]), np.array([0, 0]))),
]

env = box.Environment(surfaces, Polyhedron.fromBounds([-1.2, 0], [1.2, 2]))
atlas = box.BoxAtlas()
initial_state = box.BoxAtlasState(atlas)
initial_state.qcom = np.array([0, 1])
initial_state.vcom = np.array([-4, 0.])
initial_state.qlimb = map(np.array, [[0.75, 1], [0.25, 0], [-0.25, 0], [-0.75, 1]])

opt = BoxAtlasContactStabilization(initial_state, env)
states, inputs, contact_indicator = opt.solve()
ts = states.components[0].breaks

SolutionResult.kSolutionFound
Optimize a model with 2036 rows, 680 columns and 5274 nonzeros
Model has 228 quadratic objective terms
Coefficient statistics:
  Matrix range    [2e-03, 1e+03]
  Objective range [1e+00, 6e+02]
  Bounds range    [1e+00, 2e+00]
  RHS range       [2e-01, 1e+03]
Presolve removed 599 rows and 223 columns
Presolve time: 0.01s
Presolved: 1437 rows, 457 columns, 3665 nonzeros
Presolved model has 218 quadratic objective terms
Variable types: 381 continuous, 76 integer (76 binary)

Root relaxation: objective -2.001766e+03, 1416 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -2001.7664    0   76          - -2001.7664      -     -    0s
     0     0 -2001.7664    0   76          - -2001.7664      -     -    0s
H    0     0                    -1900.427604 -2001.7664  5.33%     -    0s
     0     2 -1978.4548    0   68 -1900.4276

In [22]:
for t in np.linspace(0, ts[-1] - 0.001, ts[-1] / 0.01):
    box.draw(vis, states(t), inputs(t), env)
    time.sleep(0.05)

In [46]:
[inputs(t).flimb[3] for t in ts[:-1]]

[array([  6.82121026e-13,   6.82121026e-13]),
 array([ 264.92602059,  -12.73515902]),
 array([ 190.71461294,   -4.78480153]),
 array([ 126.33757119,    3.82157343]),
 array([ 73.86979338,  12.97534496]),
 array([ 33.39079319,  22.47011835]),
 array([ 17.75708126,  17.75708126]),
 array([ 11.45692333,  11.45692333]),
 array([ 6.74091093,  6.74091093]),
 array([ 3.9003769,  3.9003769]),
 array([ 4.15530249,  4.15530249]),
 array([ 7.19252844,  7.19252844]),
 array([ 12.82836644,  12.82836644]),
 array([ 16.60829706,  16.60829706]),
 array([ 16.47674383,  16.47674383]),
 array([ 16.42398245,  16.42398245]),
 array([ 16.31952099,  16.31952099]),
 array([ 16.29263568,  16.29263568]),
 array([ 16.18995646,  16.18995646]),
 array([ 16.27477265,  16.27477265])]

In [47]:
[f.coeffs[0] for f in contact_indicator[3].functions]

[array([ 0.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.]),
 array([ 1.])]

In [8]:
i = 0


In [145]:
box.draw(vis, states(min(ts[i], ts[-1]-0.001)), inputs(min(ts[i], ts[-1]-0.001)), env)
if i < len(ts) - 1:
    i += 1
else:
    i = 0